# Corridor Detection Diagnostics

In [4]:
from pathlib import Path
import sys

REPO_ROOT = Path.cwd().resolve()
SRC_CANDIDATES = [
    REPO_ROOT / 'src',
    REPO_ROOT.parent / 'src',
    (REPO_ROOT / '..' / 'src').resolve(),
]

for candidate in SRC_CANDIDATES:
    if candidate.exists():
        src_path = candidate.resolve()
        break
else:
    raise RuntimeError('Could not locate the package src directory. Update the path setup cell.')

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f'Added {src_path} to sys.path')

Added /groups/spruston/home/moharb/DELTA_Behavior/src to sys.path


In [5]:

import pandas as pd
import matplotlib.pyplot as plt

from behavioral_analysis.io.json_parser import parse_json_file
from behavioral_analysis.io.dataframe_builder import extract_events_by_type
from behavioral_analysis.processing import (
    compute_corridor_artifacts,
    detect_corridors_simple,
    add_corridor_info_to_events,
)

plt.style.use('seaborn-v0_8')
from IPython.display import display


In [6]:

JSON_PATH = Path('/groups/spruston/home/moharb/DELTA_Behavior/Log BM35 2025-09-22 session 1.json')

if not JSON_PATH.exists():
    raise FileNotFoundError(f"JSON log not found: {JSON_PATH}")

print(f'Loading events from: {JSON_PATH.name}')
raw_events = parse_json_file(JSON_PATH, verbose=False)
frames = extract_events_by_type(raw_events, verbose=False)

print(f"Total events: {len(raw_events):,}")
print(f"Event types: {', '.join(sorted(frames.keys()))}")


Loading events from: Log BM35 2025-09-22 session 1.json
Total events: 241,714
Available event types: ['Cue Result', 'Cue State', 'Info', 'Lick', 'Linear Controller Settings', 'Log', 'Path Position', 'Position', 'Reward', 'Start Period']


In [7]:

corridor_info, position_with_corridors = detect_corridors_simple(
    cue_df=frames.get('Cue State'),
    position_df=frames.get('Path Position'),
    corridor_length_cm=500.0,
    verbose=True,
    cue_result_df=frames.get('Cue Result'),
)

artifacts = compute_corridor_artifacts(
    cue_state_df=frames.get('Cue State'),
    position_df=frames.get('Path Position'),
    cue_result_df=frames.get('Cue Result'),
    verbose=False,
)

cue_matches = artifacts.cue_matches
position_loops = artifacts.position_loops

updated_frames = add_corridor_info_to_events(
    dataframes={key: df.copy() for key, df in frames.items()},
    corridor_info=corridor_info,
    corridor_length_cm=500.0,
    verbose=False,
    position_df=position_with_corridors,
)

print(f"Corridors detected: {len(corridor_info)}")
print(f"Position loops: {len(position_loops)}")
print(f"Matched cues: {0 if cue_matches is None else len(cue_matches)}")


Corridors detected: 57
Position loops: 57
Matched cues: 398


,corridor_id,start_time,end_time,start_position,end_position,min_position,max_position,loop_complete,duration_ms,start_position_cm,end_position_cm,max_position_cm,trigger,num_cue_results,first_cue_time,last_cue_time,num_matched_cues,first_match_time,last_match_time
0,0,954.9052,44649.99,0,21,0.0,49988.0,True,43695.0848,0.000,0.084,199.952,first_loop,7,22231.10,45650.49,7,22231.10,45650.49
1,1,44667.7700,79615.20,55,29,29.0,49983.0,True,34947.4300,0.220,0.116,199.932,loop_reset,7,52474.25,81431.45,7,52474.25,81431.45
2,2,79630.0300,112858.50,80,1,1.0,49974.0,True,33228.4700,0.320,0.004,199.896,loop_reset,7,84919.58,113792.60,7,84919.58,113792.60
3,3,112876.3000,142935.40,33,4,4.0,49949.0,True,30059.1000,0.132,0.016,199.796,loop_reset,7,118963.00,144303.20,7,118963.00,144303.20
4,4,142950.7000,161533.60,58,12,12.0,49928.0,True,18582.9000,0.232,0.048,199.712,loop_reset,7,146804.40,162851.80,7,146804.40,162851.80


In [8]:

cols = [
    'corridor_id',
    'start_time',
    'end_time',
    'first_cue_time',
    'last_cue_time',
    'num_cue_results',
    'num_matched_cues',
    'loop_complete',
]

if not corridor_info.empty:
    display(corridor_info[cols].head(12))
else:
    print('Corridor summary unavailable.')


,corridor_id,start_time,end_time,start_position,end_position,min_position,max_position,loop_complete,duration_ms,start_position_cm,end_position_cm,max_position_cm,trigger,num_cue_results,first_cue_time,last_cue_time,num_matched_cues,first_match_time,last_match_time
0,0,9.549052e+02,44649.99,0,21,0.0,49988.0,True,43695.0848,0.000,0.084,199.952,first_loop,7,22231.10,45650.49,7,22231.10,45650.49
1,1,4.466777e+04,79615.20,55,29,29.0,49983.0,True,34947.4300,0.220,0.116,199.932,loop_reset,7,52474.25,81431.45,7,52474.25,81431.45
2,2,7.963003e+04,112858.50,80,1,1.0,49974.0,True,33228.4700,0.320,0.004,199.896,loop_reset,7,84919.58,113792.60,7,84919.58,113792.60
3,3,1.128763e+05,142935.40,33,4,4.0,49949.0,True,30059.1000,0.132,0.016,199.796,loop_reset,7,118963.00,144303.20,7,118963.00,144303.20
4,4,1.429507e+05,161533.60,58,12,12.0,49928.0,True,18582.9000,0.232,0.048,199.712,loop_reset,7,146804.40,162851.80,7,146804.40,162851.80
5,5,1.615511e+05,182952.30,95,15,15.0,49961.0,True,21401.2000,0.380,0.060,199.844,loop_reset,7,165521.50,184270.20,7,165521.50,184270.20
6,6,1.829692e+05,202669.20,80,12,12.0,49967.0,True,19700.0000,0.320,0.048,199.868,loop_reset,7,185771.40,204754.40,7,185771.40,204754.40
7,7,2.026870e+05,220050.70,66,39,39.0,49983.0,True,17363.7000,0.264,0.156,199.932,loop_reset,7,206972.30,222871.30,7,206972.30,222871.30
8,8,2.200678e+05,262839.40,98,0,0.0,49975.0,True,42771.6000,0.392,0.000,199.900,loop_reset,7,229291.50,265222.20,7,229291.50,265222.20
9,9,2.628535e+05,296634.50,24,0,0.0,49960.0,True,33781.0000,0.096,0.000,199.840,loop_reset,7,273113.40,297349.50,7,273113.40,297349.50


In [ ]:

cue_state_counts = artifacts.cue_state_with_corridors['corridor_id'].value_counts().sort_index()
cue_result_counts = artifacts.cue_result_with_corridors['corridor_id'].value_counts().sort_index()
counts_df = pd.DataFrame({
    'cue_states': cue_state_counts,
    'cue_results': cue_result_counts,
}).fillna(0).astype(int)

counts_df['all_seven'] = counts_df['cue_results'] == 7

display(counts_df.head(20))

non_standard = counts_df[counts_df['cue_results'] != 7]
if not non_standard.empty:
    print('
Corridors missing full 7 hits:')
    display(non_standard)


In [ ]:

if 'Cue Result' in updated_frames:
    display(updated_frames['Cue Result'][['corridor_id', 'cue_index', 'id', 'time', 'position', 'position_cm', 'global_position_cm']].head(14))
else:
    print('Cue Result table not available.')


In [ ]:

path_position_df = updated_frames.get('Path Position')
cue_results_df = artifacts.cue_result_with_corridors

if path_position_df is not None and not corridor_info.empty and cue_results_df is not None:
    corridor_id = 0
    loop = corridor_info[corridor_info['corridor_id'] == corridor_id].iloc[0]

    mask = (path_position_df['time'] >= loop['start_time']) & (path_position_df['time'] <= loop['end_time'])
    subset = path_position_df.loc[mask, ['time', 'position']].copy()
    subset['time_seconds'] = subset['time'] / 1000.0

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(subset['time_seconds'], subset['position'], color='steelblue', linewidth=1.5)

    hits = cue_results_df[cue_results_df['corridor_id'] == corridor_id]
    if not hits.empty:
        ax.vlines(hits['time'] / 1000.0, ymin=subset['position'].min(), ymax=subset['position'].max(),
                  colors='tomato', linestyles='--', alpha=0.6, label='Cue hits')

    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Path position (a.u.)')
    ax.set_title(f'Corridor {corridor_id}: path trajectory with cue hits')
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print('Not enough data to visualize the first corridor.')


In [ ]:

summary_cols = [
    'corridor_id',
    'start_time',
    'end_time',
    'first_cue_time',
    'last_cue_time',
    'num_cue_results',
    'loop_complete',
]

if not corridor_info.empty:
    display(corridor_info[summary_cols].tail(10))
else:
    print('Corridor summary unavailable.')
